In [ ]:
import os, sys, subprocess, gc, traceback, torch
from pathlib import Path
import soundfile as sf
import numpy as np
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

def log(msg):
    ts = datetime.now().strftime('%H:%M:%S')
    line = f'[{ts}] {msg}'
    print(line, flush=True)
    with open('kaggle_remote.log', 'a') as f:
        f.write(line + '\n')

log('Checking environment...')
try:
    if subprocess.run('sox --version', shell=True, capture_output=True).returncode != 0:
        log('Installing system sox and upgrading transformers...')
        subprocess.run('apt-get update -q && apt-get install -y -q sox libsox-fmt-all', shell=True)
        subprocess.run('pip install -q -U sox qwen-tts==0.1.1 "transformers>=4.48.0"', shell=True)
    else:
        log('Sox already present. Ensuring python packages...')
        subprocess.run('pip install -q -U qwen-tts==0.1.1 "transformers>=4.48.0"', shell=True)
    
    from qwen_tts import Qwen3TTSModel
    log('Setup complete (Imports done).')
except Exception as e:
    log(f'Setup warning: {e}. Attempting recovery full install...')
    subprocess.run('pip install -q -U qwen-tts==0.1.1 sox "transformers>=4.48.0" --no-cache-dir', shell=True)
    from qwen_tts import Qwen3TTSModel
    log('Setup complete (Recovery Imports done).')

# Parameters
text = "Wait it out end to end that it runs fully and generates&downloads the qwen audio file. No more excuses. I am Qwen on T4."
voice = 'Ryan'
output_file = "final_victory.wav"

log(f'Loading Qwen model...')
try:
    model = Qwen3TTSModel.from_pretrained(
        'Qwen/Qwen3-TTS-12Hz-0.6B-CustomVoice',
        torch_dtype=torch.float32,
        device_map='auto'
    )
    
    log(f'Synthesizing...')
    # Synthesis block
    with torch.inference_mode():
        audio_array, sample_rate = model.generate_custom_voice(
            text=text,
            speaker=voice,
            language='english',
            do_sample=False,
            subtalker_dosample=False
        )
    
    if audio_array is not None and len(audio_array) > 0:
        sf.write(output_file, audio_array, int(sample_rate))
        log(f'SUCCESS: Audio saved to {output_file}')
    else:
        log('ERROR: Empty result')
except Exception as e:
    log(f'ERROR: {str(e)}')
    traceback.print_exc()

del model
torch.cuda.empty_cache()
gc.collect()
log('Cleanup done.')
